In [8]:
import collections
import os, csv, random, math
import bz2
import numpy as np
import tensorflow as tf
import nltk
import time
from math import ceil
from scipy.sparse import lil_matrix

In [12]:
filename = '../data/wikipedia2text-extracted.txt.bz2'
vocabulary_size = 5000 # origin 50000

batch_size = 24 # origin 128
embedding_size = 24 # origin 128
window_size = 2 # origin 4

# We pick a random validation set to sample nearest neighbors
valid_size = 16 # Random set of words to evaluate similarity on.
# We sample valid datapoints randomly from a large window without always being deterministic
valid_window = 50
# When selecting valid examples, we select some of the most frequent words as well as
# some moderately rare words as well
valid_examples = np.array(random.sample(range(valid_window), valid_size))
valid_examples = np.append(valid_examples,random.sample(range(1000, 1000+valid_window), valid_size),axis=0)
num_sampled = 32 # Number of negative examples to sample.

num_steps = 100001 # origin 100001

skip_window = 2 # How many words to consider left and right.
epsilon = 1 # used for the stability of log in the loss function

In [4]:
def read_data(filename):    
    with bz2.BZ2File(filename) as f:        
        data = []
        file_size = os.stat(filename).st_size
        chunk_size = 1024 * 1024 # reading 1 MB at a time as the dataset is moderately large
        print('Reading data...')

        for i in range(ceil(file_size//chunk_size)+1):        
            print('...%02d/%02d'%(i,ceil(file_size//chunk_size)),
                  time.strftime("%H:%M:%S", time.localtime()))
            bytes_to_read = min(chunk_size,file_size-(i*chunk_size))
            file_string = f.read(bytes_to_read).decode('utf-8')
            file_string = file_string.lower()
            # tokenizes a string to words residing in a list
            file_string = nltk.word_tokenize(file_string)
            data.extend(file_string)
    
    return data

tStart = time.time()
words = read_data(filename)
token_count = len(words)
print('Data size %d' % token_count)
print('Example words (start): ',words[:10])
print('Example words (end): ',words[-10:])
tEnd = time.time()
print('Cost %.2f seconds' % (tEnd - tStart))

Reading data...
...00/17 20:09:59
...01/17 20:10:00
...02/17 20:10:01
...03/17 20:10:02
...04/17 20:10:04
...05/17 20:10:05
...06/17 20:10:06
...07/17 20:10:07
...08/17 20:10:09
...09/17 20:10:10
...10/17 20:10:11
...11/17 20:10:12
...12/17 20:10:14
...13/17 20:10:15
...14/17 20:10:16
...15/17 20:10:18
...16/17 20:10:20
...17/17 20:10:22
Data size 3361213
Example words (start):  ['propaganda', 'is', 'a', 'concerted', 'set', 'of', 'messages', 'aimed', 'at', 'influencing']
Example words (end):  ['favorable', 'long-term', 'outcomes', 'for', 'around', 'half', 'of', 'those', 'diagnosed', 'with']
Cost 23.93 seconds


In [5]:
def build_dataset(words):
    count = [['UNK', -1]]  
    count.extend(collections.Counter(words).most_common(vocabulary_size - 1))
    dictionary = dict()    
    for word, _ in count:
        dictionary[word] = len(dictionary)    
    data = list()
    unk_count = 0 
    for word in words:
        if word in dictionary:
            index = dictionary[word]
        else:
            index = 0  # dictionary['UNK']
            unk_count = unk_count + 1
        data.append(index)
    count[0][1] = unk_count
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys())) 
    assert len(dictionary) == vocabulary_size
    
    return data, count, dictionary, reverse_dictionary
        
data, count, dictionary, reverse_dictionary = build_dataset(words)
print('Most common words (+UNK)', count[:5])
print('Sample data', data[:10])
del words  # Hint to reduce memory.

Most common words (+UNK) [['UNK', 500328], ('the', 226893), (',', 184013), ('.', 120919), ('of', 116323)]
Sample data [1721, 9, 8, 0, 223, 4, 0, 4459, 26, 0]


In [6]:
data_index = 0

def generate_batch(batch_size, window_size):
    # data_index is updated by 1 everytime we read a data point
    global data_index
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    weights = np.ndarray(shape=(batch_size), dtype=np.float32)
    span = 2 * window_size + 1
    buffer = collections.deque(maxlen=span)
    for _ in range(span):
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)     
    num_samples = 2*window_size 
    for i in range(batch_size // num_samples):    
        k=0        
        for j in list(range(window_size))+list(range(window_size+1,2*window_size+1)):
            batch[i*num_samples+k] = buffer[window_size]
            labels[i*num_samples+k, 0] = buffer[j]
            weights[i*num_samples+k] = abs(1.0/(j-window_size))
            k+=1
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
    return batch, labels, weights

print('data:', [reverse_dictionary[di] for di in data[:8]])
for window_size in [2, 4]:
    data_index = 0
    batch, labels, weights = generate_batch(batch_size=8, window_size=window_size)
    print('\nwith window_size = %d:' % window_size)
    print('    batch:', [reverse_dictionary[bi] for bi in batch])
    print('    labels:', [reverse_dictionary[li] for li in labels.reshape(8)])
    print('    weights:', [w for w in weights])

data: ['propaganda', 'is', 'a', 'UNK', 'set', 'of', 'UNK', 'aimed']

with window_size = 2:
    batch: ['a', 'a', 'a', 'a', 'UNK', 'UNK', 'UNK', 'UNK']
    labels: ['propaganda', 'is', 'UNK', 'set', 'is', 'a', 'set', 'of']
    weights: [0.5, 1.0, 1.0, 0.5, 0.5, 1.0, 1.0, 0.5]

with window_size = 4:
    batch: ['set', 'set', 'set', 'set', 'set', 'set', 'set', 'set']
    labels: ['propaganda', 'is', 'a', 'UNK', 'of', 'UNK', 'aimed', 'at']
    weights: [0.25, 0.33333334, 0.5, 1.0, 1.0, 0.5, 0.33333334, 0.25]


## Creating the Word Co-Occurance Matrix
Why GloVe shine above context window based method is that it employs global statistics of the corpus in to the model (according to authors). This is done by using information from the word co-occurance matrix to optimize the word vectors. Basically, the X(i,j) entry of the co-occurance matrix says how frequent word i to appear near j. We also use a weighting mechanishm to give more weight to words close together than to ones further-apart (from experiments section of the paper).

In [11]:
cooc_data_index = 0
dataset_size = len(data) # We iterate through the full text

# The sparse matrix that stores the word co-occurences
cooc_mat = lil_matrix((vocabulary_size, vocabulary_size), dtype=np.float32)

print(cooc_mat.shape)
def generate_cooc(batch_size,skip_window):
    data_index = 0
    print('Running %d iterations to compute the co-occurance matrix'%(dataset_size//batch_size))
    for i in range(dataset_size//batch_size):
        # Printing progress
        if i>0 and i%100000==0:
            print('\tFinished %d iterations'%i)
        batch, labels, weights = generate_batch(batch_size, skip_window)
        labels = labels.reshape(-1)  
        for inp,lbl,w in zip(batch,labels,weights):            
            cooc_mat[inp,lbl] += (1.0*w)

tStart = time.time()
# Generate the matrix
generate_cooc(8,skip_window)    
print('Sample chunks of co-occurance matrix')
# Basically calculates the highest cooccurance of several chosen word
for i in range(10):
    idx_target = i    
    # get the ith row of the sparse matrix and make it dense
    ith_row = cooc_mat.getrow(idx_target)     
    ith_row_dense = ith_row.toarray('C').reshape(-1)            
    # select target words only with a reasonable words around it.
    while np.sum(ith_row_dense)<10 or np.sum(ith_row_dense)>50000:
        # Choose a random word
        idx_target = np.random.randint(0,vocabulary_size)
        ith_row = cooc_mat.getrow(idx_target) 
        ith_row_dense = ith_row.toarray('C').reshape(-1)    
        
    print('\nTarget Word: "%s"'%reverse_dictionary[idx_target], time.strftime('%H:%M:%S',time.localtime()))
        
    sort_indices = np.argsort(ith_row_dense).reshape(-1) # indices with highest count of ith_row_dense
    sort_indices = np.flip(sort_indices,axis=0) # reverse the array (to get max values to the start)

    # printing several context words to make sure cooc_mat is correct
    print('Context word:',end='')
    for j in range(10):        
        idx_context = sort_indices[j]       
        print('"%s"(id:%d,count:%.2f), '%(reverse_dictionary[idx_context],idx_context,ith_row_dense[idx_context]),end='')
    print()
tEnd = time.time()
print('Cost %.2f seconds'%(tEnd-tStart))

(5000, 5000)
Running 420151 iterations to compute the co-occurance matrix
	Finished 100000 iterations
	Finished 200000 iterations
	Finished 300000 iterations
	Finished 400000 iterations
Sample chunks of co-occurance matrix

Target Word: "lives" 20:30:16
Context word:"of"(id:4,count:18.50), "their"(id:41,count:15.50), "the"(id:1,count:14.50), "UNK"(id:0,count:13.00), "."(id:3,count:11.00), "and"(id:5,count:8.50), "in"(id:6,count:8.50), ","(id:2,count:7.50), "population"(id:123,count:4.00), "on"(id:18,count:2.50), 

Target Word: "surface" 20:30:16
Context word:"the"(id:1,count:48.50), "UNK"(id:0,count:33.00), "."(id:3,count:22.50), "of"(id:4,count:22.00), ","(id:2,count:14.00), "'s"(id:19,count:11.50), "to"(id:7,count:9.00), "a"(id:8,count:8.00), "and"(id:5,count:6.00), "on"(id:18,count:5.00), 

Target Word: "workers" 20:30:16
Context word:"UNK"(id:0,count:23.00), ","(id:2,count:16.00), "of"(id:4,count:11.50), "."(id:3,count:10.00), "the"(id:1,count:8.00), "from"(id:21,count:5.00), "'"(i

## GloVe Algorithm

In [13]:
tf.reset_default_graph()
train_dataset = tf.placeholder(tf.int32, shape=[batch_size])
train_labels = tf.placeholder(tf.int32, shape=[batch_size])
valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

in_embeddings = tf.Variable(tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0),name='embeddings')
in_bias_embeddings = tf.Variable(tf.random_uniform([vocabulary_size],0.0,0.01,dtype=tf.float32),name='embeddings_bias')
out_embeddings = tf.Variable(tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0),name='embeddings')
out_bias_embeddings = tf.Variable(tf.random_uniform([vocabulary_size],0.0,0.01,dtype=tf.float32),name='embeddings_bias')

embed_in = tf.nn.embedding_lookup(in_embeddings, train_dataset)
embed_out = tf.nn.embedding_lookup(out_embeddings, train_labels)
embed_bias_in = tf.nn.embedding_lookup(in_bias_embeddings,train_dataset)
embed_bias_out = tf.nn.embedding_lookup(out_bias_embeddings,train_labels)
# weights used in the cost function
weights_x = tf.placeholder(tf.float32,shape=[batch_size],name='weights_x') 
# Cooccurence value for that position
x_ij = tf.placeholder(tf.float32,shape=[batch_size],name='x_ij')

loss = tf.reduce_mean(
    weights_x * (tf.reduce_sum(embed_in*embed_out,axis=1) + embed_bias_in + embed_bias_out - tf.log(epsilon+x_ij))**2)

## Calculating Word Similarities
embeddings = (in_embeddings + out_embeddings)/2.0
norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keepdims=True))
normalized_embeddings = embeddings / norm
valid_embeddings = tf.nn.embedding_lookup(
normalized_embeddings, valid_dataset)
similarity = tf.matmul(valid_embeddings, tf.transpose(normalized_embeddings))

optimizer = tf.train.AdagradOptimizer(1.0).minimize(loss)

In [16]:
glove_loss = []
average_loss = 0
tStart= time.time()
with tf.Session(config=tf.ConfigProto(allow_soft_placement=True)) as session:    
    tf.global_variables_initializer().run()
    print('Initialized',time.strftime('%H:%M:%S',time.localtime()))    
    for step in range(num_steps):        
        batch_data, batch_labels, batch_weights = generate_batch(batch_size, skip_window)         
        batch_weights = [] # weighting used in the loss function
        batch_xij = [] # weighted frequency of finding i near j        
        # Compute the weights for each datapoint in the batch
        for inp,lbl in zip(batch_data,batch_labels.reshape(-1)):     
            point_weight = (cooc_mat[inp,lbl]/100.0)**0.75 if cooc_mat[inp,lbl]<100.0 else 1.0 
            batch_weights.append(point_weight)
            batch_xij.append(cooc_mat[inp,lbl])
        batch_weights = np.clip(batch_weights,-100,1)
        batch_xij = np.asarray(batch_xij)        
        # Populate the feed_dict and run the optimizer (minimize loss)
        # and compute the loss. Specifically we provide
        # train_dataset/train_labels: training inputs and training labels
        # weights_x: measures the importance of a data point with respect to how much those two words co-occur
        # x_ij: co-occurence matrix value for the row and column denoted by the words in a datapoint
        feed_dict = {train_dataset : batch_data.reshape(-1), train_labels : batch_labels.reshape(-1),
                    weights_x:batch_weights,x_ij:batch_xij}
        _, l = session.run([optimizer, loss], feed_dict=feed_dict)
        
        # Update the average loss variable
        average_loss += l
        if step % 2000 == 0:
            if step > 0:
                average_loss = average_loss / 2000          
            print('Average loss at step %d: %f' % (step, average_loss))
            glove_loss.append(average_loss)
            average_loss = 0        
        if step % 10000 == 0:
            print(time.strftime('%H:%M:%S',time.localtime()))
            sim = similarity.eval()
            for i in range(valid_size):
                valid_word = reverse_dictionary[valid_examples[i]]
                top_k = 8 # number of nearest neighbors
                nearest = (-sim[i, :]).argsort()[1:top_k+1]
                log = 'Nearest to %s:' % valid_word
                for k in range(top_k):
                    close_word = reverse_dictionary[nearest[k]]
                    log = '%s %s,' % (log, close_word)
                print('valid_size%02d : '%i,log)            
    final_embeddings = normalized_embeddings.eval()
    
tEnd = time.time()
print('Cost %.2f seconds'%(tEnd-tStart))

Initialized 20:48:31
Average loss at step 0: 31.904291
20:48:32
valid_size00 :  Nearest to which: felix, rail, earth, church, biological, horizontal, hero, provincial,
valid_size01 :  Nearest to as: supported, devoted, test, claims, grand, japan, develop, decline,
valid_size02 :  Nearest to .: UNK, it, decade, railway, or, takes, description, particles,
valid_size03 :  Nearest to were: planets, released, -, neighboring, still, votes, gaining, data,
valid_size04 :  Nearest to an: reasons, precipitation, seems, symbols, experiments, find, station, heaven,
valid_size05 :  Nearest to of: decay, blue, base, communication, castle, column, religious, competition,
valid_size06 :  Nearest to from: tested, techniques, egyptians, ran, islamic, reading, 1998, assembled,
valid_size07 :  Nearest to and: liberalism, d, 10, army, experts, beat, amsterdam, ',
valid_size08 :  Nearest to city: temple, milk, guns, programme, promised, victor, millennium, satellite,
valid_size09 :  Nearest to most: image, 

Average loss at step 52000: 0.027225
Average loss at step 54000: 0.028118
Average loss at step 56000: 0.027580
Average loss at step 58000: 0.028751
Average loss at step 60000: 0.028986
20:49:25
valid_size00 :  Nearest to which: was, he, it, ,, but, is, when, also,
valid_size01 :  Nearest to as: such, a, well, an, known, ``, UNK, with,
valid_size02 :  Nearest to .: the, in, UNK, of, ,, and, for, ),
valid_size03 :  Nearest to were: was, are, may, is, had, have, would, by,
valid_size04 :  Nearest to an: a, with, as, UNK, ,, and, for, to,
valid_size05 :  Nearest to of: the, ., 's, UNK, in, state, a, and,
valid_size06 :  Nearest to from: to, on, UNK, the, by, for, ., in,
valid_size07 :  Nearest to and: ,, UNK, the, ., in, ;, for, with,
valid_size08 :  Nearest to city: region, country, state, first, during, capital, government, 's,
valid_size09 :  Nearest to most: state, of, all, population, area, parts, modern, rest,
valid_size10 :  Nearest to he: when, she, which, was, they, it, had, were,